In [47]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import Dataset
torch.backends.cudnn.benchmark=True

In [48]:
from sklearn.model_selection import train_test_split
data = np.load('data.npy').astype('float32')
labels = np.load('labels.npy').astype('float32')
print("data : ", data.shape)
print("labels : ", labels.shape)

x_train, x_test, y_train, y_test = train_test_split(data,labels,test_size=0.15)
print("x_train : ",x_train.shape)
print("x_test : ",x_test.shape)


data :  (45222, 67)
labels :  (45222,)
x_train :  (38438, 67)
x_test :  (6784, 67)


In [49]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(67, 128)
        self.relu1 = nn.ReLU()
        self.dout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(64, 32)
        self.prelu = nn.PReLU(1)
        self.out = nn.Linear(32, 1)
        self.out_act = nn.Sigmoid()
        
    def forward(self, input_):
        inp = self.fc1(input_)
        x = self.relu1(inp)
        x = self.dout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dout2(x)
        x = self.fc3(x)
        x = self.prelu(x)
        x = self.out(x)
        y = self.out_act(x)
        return y
    
# net = Net().to("cuda:0")
net = Net()
opt = optim.Adam(net.parameters(), lr=0.01, betas=(0.9, 0.999))
criterion = nn.BCELoss()

In [50]:
def train_epoch(model, opt, criterion, batch_size=128):
    model.train()
    losses = []
    for beg_i in range(0, x_train.shape[0], batch_size):
        x_batch = x_train[i: i + batch_size, :]
        y_batch = y_train[i: i + batch_size]
        x_batch = torch.from_numpy(x_batch)
        y_batch = torch.from_numpy(y_batch)
#         print(y_batch.dtype)
        opt.zero_grad()
        y_hat = net(x_batch)
        loss = criterion(y_hat, y_batch)
        loss.backward()
        opt.step()
        losses.append(loss.cpu().data.numpy())
    return np.mean(losses)

In [51]:
e_losses = []
num_epochs =30
for e in range(num_epochs):
    loss = train_epoch(net, opt, criterion)
    print(loss)
    e_losses.append(loss)
    
plt.plot(e_losses)

NameError: name 'i' is not defined

In [ ]:
def classification(X, y):
    prediction = np.zeros((X.shape[0],1)) 
    # Final Model Output
    probabilities=net(torch.from_numpy(X))
    
    # Predictions
    for i in range(0, probabilities.shape[0]):    
            prediction[i,0] = 1 if probabilities[i,0] > 0.5 else 0
    print("Accuracy  = "  + str(np.sum((prediction.flatten() == y.flatten())*100/X.shape[0]))+"%")

In [ ]:
classification(x_test,y_test)